In [2]:
!pip install torchvision
!pip install torch

In [1]:
import json
import torchvision
import torch
import torch.nn as nn
import torchvision.transforms as T
import os
import xml.etree.ElementTree as ET
from PIL import Image,ImageDraw,ImageOps,ExifTags
from IPython.display import display
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import numpy as np

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
os.chdir('../datasets/datasets/')
from mapeval import voc_eval

: 

In [ ]:
def rotateImage(img):
  image_exif = img._getexif()
  image_orientation = image_exif[274]
  if image_orientation == 3:
    rotated = img.rotate(180)
  if image_orientation == 6:
    rotated = img.rotate(-90)
  if image_orientation == 8:
    rotated = img.rotate(90)
  return rotated 

In [3]:
img = Image.open('carros/2247813774_e722d94c4b_n.jpg')
#img = rotateImage(img)
img = ImageOps.exif_transpose(img)

NameError: name 'Image' is not defined

In [ ]:
img.resize((600, 600))

In [ ]:
# dados rotulados de https://www.makesense.ai/
h_files = {}
with open('/content/drive/MyDrive/class_ia/labels.csv') as csv_file:
    for line in csv_file:
        v = line.split(',')
        class_att = v[0]
        x1,y1,x2,y2 = [int(x) for x in v[1:5]]
        img_file = v[5]
        if img_file not in h_files.keys():
            h_files[img_file] = {'boxes':[],'labels':[]}
        h_files[img_file]['boxes'].append((x1,y1,x2,y2)) 
        h_files[img_file]['labels'].append(class_att)

for img_file in h_files.keys():
    print(img_file)
    print(h_files[img_file]['boxes'])
    print(h_files[img_file]['labels'])

In [ ]:
class MyDataset():
    def __init__(self,transforms = None):
        self.data = []
        self.transforms = transforms
        self.target_names = ['blanck']
        self.htarget_names = {'blanck':0,'vaso flor 1':1,'vaso flor 2':2}
        self.read_csv()

    def get_label_id(self,name):
        if name not in self.htarget_names:
            self.htarget_names[name] = len(self.target_names)
            self.target_names.append(name)
        return self.htarget_names[name]
    
    def read_csv(self):
        h_files = {}
        with open('labels.csv') as csv_file:
            for line in csv_file:
                v = line.split(',')
                class_att = v[0]
                x1,y1,width,height = [int(x) for x in v[1:5]]
                x2 = x1 + width
                y2 = y1 + height
                img_file = v[5]
                if img_file not in h_files.keys():
                    h_files[img_file] = {'boxes':[],'labels':[]}
                h_files[img_file]['boxes'].append((x1,y1,x2,y2)) 
                h_files[img_file]['labels'].append(self.get_label_id(class_att))

        for img_file in h_files.keys():
            h = {}
            h['file_img'] = 'imagens/'+img_file
            h['labels'] = h_files[img_file]['labels']
            h['boxes']  = h_files[img_file]['boxes']
            self.data.append(h)
    
    def __getitem__(self,i):
        img   = Image.open(self.data[i]['file_img'])
        img   = ImageOps.exif_transpose(img)
        img   = img.convert("RGB")
        boxes = torch.tensor(self.data[i]['boxes'])
        if self.transforms != None:
            img,boxes = self.transforms(img,boxes)
        r = dict()
        r['boxes']   = boxes
        r['labels']  = torch.tensor(self.data[i]['labels'])
        return img,r
    def __len__(self):
        return len(self.data)

In [ ]:
def resize(img,boxes,size):
    w, h = img.size
    ow, oh = size
    sw = float(ow) / w
    sh = float(oh) / h
    img = img.resize((ow,oh), Image.BILINEAR)
    boxes = boxes * torch.tensor([sw,sh,sw,sh])
    return img, boxes

In [ ]:
size = (300,300)
def transform_data(img,boxes):
    img,boxes = resize(img,boxes,size)
    img = T.Compose([
          T.ToTensor(),
          #(image - mean) / std
          T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))          
    ])(img)
    return img,boxes

In [ ]:
data = MyDataset(transforms=transform_data)

In [ ]:
len(data)

In [ ]:
n = len(data)
n_treino = int(0.7*n)
n_teste  = n-n_treino

In [ ]:
n,n_treino,n_teste

In [ ]:
ds_treino,ds_teste = torch.utils.data.random_split(data,(n_treino,n_teste))

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))
dl_treino = torch.utils.data.DataLoader(ds_treino,batch_size = 8,collate_fn=collate_fn)
dl_teste  = torch.utils.data.DataLoader(ds_teste,batch_size = 12,collate_fn=collate_fn)

In [ ]:
imgs,targets = next(iter(dl_treino))
imgs_teste,targets_teste = next(iter(dl_teste))

In [ ]:
len(imgs)

In [ ]:
data[0]

In [ ]:
nview = ImageOps.exif_transpose(Image.open(data.data[0]['file_img']))

In [ ]:
data.data[0]['boxes']

In [ ]:
def draw_boxes(img,boxes,labels):
    imdraw = ImageDraw.Draw(img)
    for (box,label) in zip(boxes,labels):
        box = list(box)
        imdraw.rectangle(box,outline='red')
        text = "%d"%(label)
        imdraw.text((box[0],box[1]),text,fill='red')
    display(img)

In [ ]:
draw_boxes(nview,data.data[0]['boxes'],data.data[0]['labels'])

In [ ]:
nview = T.ToPILImage()(imgs[0]*torch.Tensor([0.229,0.224,0.225]).view(3,1,1)+torch.Tensor([0.485,0.456,0.406]).view(3,1,1))

In [ ]:
draw_boxes(nview,targets[0]['boxes'],targets[0]['labels'])

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
model

In [ ]:
# Fine tunning
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features,3)

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

In [ ]:
model.to(device)

In [ ]:
def train(epoch):
    model.train()
    bloss=[]
    for images,targets in dl_treino:
        images = list(image.to(device) for image in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]
        
        loss_dict = model(images,targets)
        losses = sum(loss for loss in loss_dict.values())
        
        opt.zero_grad()
        losses.backward()
        opt.step()
        
        for loss in loss_dict.keys():
            print("%.10s %4.3f"%(loss,loss_dict[loss].item()))
        print("Total Loss %4.3f\n"%(losses))
        bloss.append(losses.item())
    
    print("\nEPOCH %d LR %5.5f\n"%(epoch,opt.param_groups[0]['lr']))

In [ ]:
def evaluate(epoch):
    model.eval()
    pred_boxes = []
    pred_labels = []
    pred_scores = []
    gt_boxes = []
    gt_labels = []
    lmap = []
    lap  = []
    with torch.no_grad():
        for images,targets in dl_teste:
            images = list(image.to(device) for image in images)
            pred   = model(images)
            for i in range(len(targets)):
                gt_boxes.append(targets[i]['boxes'])
                gt_labels.append(targets[i]['labels'])
                pred_boxes.append(pred[i]['boxes'].cpu())
                pred_labels.append(pred[i]['labels'].cpu())
                pred_scores.append(pred[i]['scores'].cpu())
                r = voc_eval(pred_boxes, pred_labels, pred_scores,
                gt_boxes, gt_labels)
                lmap.append(r['map'])
    #print(np.mean(lmap))
    print("mAP %.2f\n"%(np.mean(lmap)))
    return np.mean(lmap)

In [ ]:
best_map = 0.0

In [ ]:
for epoch in range(10):
    train(epoch)
    map = evaluate(epoch)
    lr_scheduler.step(1.0-map)
    if map > best_map:
        best_map = map
        torch.save(model,'best_map_labelbox.pth')
        print('saving model')

In [ ]:
model = torch.load('best_map_labelbox.pth')

In [ ]:
image_pos = -1

In [ ]:
nview = T.ToPILImage()(imgs[image_pos]*torch.Tensor([0.229,0.224,0.225]).view(3,1,1)+torch.Tensor([0.485,0.456,0.406]).view(3,1,1))

In [ ]:
model.eval()

In [ ]:
pred   = model(imgs[image_pos].view([1,3,300,300]).to(device))

In [ ]:
pred

In [ ]:
pred[0]['boxes'][:2],pred[0]['labels'][:2]

In [ ]:
draw_boxes(nview,pred[0]['boxes'][:2],pred[0]['labels'][:2])